In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t

import ft_digital_trade.src.utils.read_data as read_utils
import ft_digital_trade.src.utils.clean_utils as clean_utils
import ft_digital_trade.src.utils.calculation_utils as calc_utils
import ft_digital_trade.src.utils.plot_utils as plot_utils

client = bigquery.Client()

In [ ]:
# Calculating Visa marketshare drop-off using change in cardholders over time
# Looks at how total number of UK cardholders in the dataset changes over time to scale each category of spend

cardholders = '''SELECT time_period_value, sum(cardholders) as total_cardholders
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  GROUP BY time_period_value
  ORDER BY time_period_value ASC
'''
cardholders_total = bq.read_bq_table_sql(client, cardholders)
#cardholders_total

base_cardholders = cardholders_total['total_cardholders'].iloc[0]
#base_cardholder

cardholders_total['Change from Base'] = (base_cardholders / cardholders_total['total_cardholders'])
cardholders_total # Change from Base column can now be multiplied against each quarter's spend values to adjust the spend for Visa's marketshare

In [ ]:
# Summing everything up on a yearly basis
yearly_cardholders = cardholders_total
yearly_cardholders['Year'] = yearly_cardholders['time_period_value'].str[:4]
yearly_cardholders = yearly_cardholders.groupby(['Year'])['total_cardholders'].sum().reset_index()

yearly_base_cardholders = yearly_cardholders['total_cardholders'].iloc[0]
yearly_cardholders['Change from Base'] = (yearly_base_cardholders / yearly_cardholders['total_cardholders'])

#yearly_cardholders

In [ ]:
# 1. Online vs Face-to-Face Spending

In [ ]:
# Total spend by UK cardholders
total_spend = '''SELECT time_period_value, sum(spend) as total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
spend_total = bq.read_bq_table_sql(client, total_spend)
#spend_total

In [ ]:
# Total online spend by UK cardholders
total_online_spend = '''SELECT time_period_value, sum(spend) as total_online_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
online_spend = bq.read_bq_table_sql(client, total_online_spend)
#online_spend.head()

In [ ]:
# Total online spend by UK cardholders
total_f2f_spend = '''SELECT time_period_value, sum(spend) as total_f2f_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'Face to Face'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
f2f_spend = bq.read_bq_table_sql(client, total_f2f_spend)
#f2f_spend.head()

In [ ]:
# Merge tables together and check if sum of online+f2f = total (it does)
online_vs_f2f = pd.merge(online_spend, f2f_spend, on='time_period_value')
online_vs_f2f = pd.merge(online_vs_f2f, spend_total, on='time_period_value')
online_vs_f2f['Total_Spend'] = online_vs_f2f['total_online_spend'] + online_vs_f2f['total_f2f_spend']
online_vs_f2f['check'] = online_vs_f2f['Total_Spend'] - online_vs_f2f['total_spend']
#online_vs_f2f

In [ ]:
# Drop the additional total spend figure and check columns
online_vs_f2f = online_vs_f2f.drop(columns=['total_spend', 'check'])
#online_vs_f2f

In [ ]:
# Calculating the adjusted spend values for each category
adjusted_spend = online_vs_f2f
adjusted_spend['adjusted_total_online_spend'] = adjusted_spend['total_online_spend'] * cardholders_total['Change from Base']
adjusted_spend['adjusted_total_f2f_spend'] = adjusted_spend['total_f2f_spend'] * cardholders_total['Change from Base']
adjusted_spend['adjusted_total_spend'] = adjusted_spend['Total_Spend'] * cardholders_total['Change from Base']

#adjusted_spend

In [ ]:
# Dropping the un-adjusted values to clean the table
adjusted_spend = adjusted_spend.drop(columns=['total_online_spend', 'total_f2f_spend', 'Total_Spend'])
adjusted_spend.to_csv('test.csv')

In [ ]:
# Calculating the indices of spend for each category of spend
online_vs_f2f_index = adjusted_spend

# Defining a function to calculate the average spend in 2019 for each category to use as the base value

def average_spend(df, column_name, year=2019):
    # Convert year to string for comparison
    year_str = str(year)
    
    # Filter rows where 'time_period_value' starts with the year
    filtered_df = df[df['time_period_value'].astype(str).str.startswith(year_str)]
    
    # Calculate and return the average
    return filtered_df[column_name].mean()


# Calculating the indexed spend value for each type of spend
online_vs_f2f_index['online_spend_index'] = (online_vs_f2f_index['adjusted_total_online_spend'] / average_spend(online_vs_f2f_index, 'adjusted_total_online_spend')) * 100
online_vs_f2f_index['f2f_spend_index'] = (online_vs_f2f_index['adjusted_total_f2f_spend'] / average_spend(online_vs_f2f_index, 'adjusted_total_f2f_spend')) * 100
online_vs_f2f_index['total_spend_index'] = (online_vs_f2f_index['adjusted_total_spend'] / average_spend(online_vs_f2f_index, 'adjusted_total_spend')) * 100

#average_spend(online_vs_f2f_index, 'adjusted_total_online_spend')
online_vs_f2f_index

In [ ]:
# Dropping additional columns so it contains just data used for graph
online_vs_f2f_index = online_vs_f2f_index.drop(columns=['adjusted_total_online_spend', 'adjusted_total_f2f_spend', 'adjusted_total_spend'])
online_vs_f2f_index

In [ ]:
# Line chart for the first section's graph
fig1 = px.line(
        online_vs_f2f_index,
        x="time_period_value",
        y=["online_spend_index", "f2f_spend_index", "total_spend_index"],
    )
fig1

In [ ]:
# Difference between 202506 and 202106
starting_month = "202106"
finishing_month = "202506"

online_start = online_vs_f2f_index


# Extract values directly
online_start_value = online_vs_f2f_index.loc[online_vs_f2f_index['time_period_value'] == starting_month, 'f2f_spend_index'].values[0]
online_end_value = online_vs_f2f_index.loc[online_vs_f2f_index['time_period_value'] == finishing_month, 'f2f_spend_index'].values[0]

difference = online_end_value - online_start_value
difference

In [ ]:
# 2. International vs Domestic Online Spend

In [ ]:
# Total spend in the UK by UK cardholders
total_spend_domestic = '''SELECT time_period_value, sum(spend) as total_spend_domestic
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All' 
  AND destination_country = 'UNITED KINGDOM'
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
spend_domestic = bq.read_bq_table_sql(client, total_spend_domestic)
#spend_domestic.head()

In [ ]:
# Total spend in foreign countries by UK cardholders
total_spend_abroad = '''SELECT time_period_value, sum(spend) as total_spend_abroad
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter'
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
spend_abroad = bq.read_bq_table_sql(client, total_spend_abroad)
#spend_abroad.head()

In [ ]:
# Total online spend in UK by UK cardholders
total_online_spend_domestic = '''SELECT time_period_value, sum(spend) as total_online_spend_domestic
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country = 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
online_spend_domestic = bq.read_bq_table_sql(client, total_online_spend_domestic)
#online_spend_domestic.head()

In [ ]:
# Total online spend in foreign countries by UK cardholders
total_online_spend_abroad = '''SELECT time_period_value, sum(spend) as total_online_spend_abroad
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All'
  AND mcc = 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
online_spend_abroad = bq.read_bq_table_sql(client, total_online_spend_abroad)
#online_spend_abroad.head()

In [ ]:
# Merging all together into one table
int_vs_dom_spend = pd.merge(spend_domestic, spend_abroad, on='time_period_value')
int_vs_dom_spend = pd.merge(int_vs_dom_spend, online_spend_domestic, on='time_period_value')
int_vs_dom_spend = pd.merge(int_vs_dom_spend, online_spend_abroad, on='time_period_value')

#int_vs_dom_spend

In [ ]:
adjusted_int_vs_dom_spend = int_vs_dom_spend
adjusted_int_vs_dom_spend['adjusted_total_domestic_spend'] = adjusted_int_vs_dom_spend['total_spend_domestic'] * cardholders_total['Change from Base']
adjusted_int_vs_dom_spend['adjusted_total_abroad_spend'] = adjusted_int_vs_dom_spend['total_spend_abroad'] * cardholders_total['Change from Base']
adjusted_int_vs_dom_spend['adjusted_online_domestic_spend'] = adjusted_int_vs_dom_spend['total_online_spend_domestic'] * cardholders_total['Change from Base']
adjusted_int_vs_dom_spend['adjusted_online_abroad_spend'] = adjusted_int_vs_dom_spend['total_online_spend_abroad'] * cardholders_total['Change from Base']

adjusted_int_vs_dom_spend

In [ ]:
# Calculating the proportions of each categories spend that is online
int_vs_dom_ratios = adjusted_int_vs_dom_spend
int_vs_dom_ratios['domestic_online_ratio'] = (int_vs_dom_ratios['adjusted_online_domestic_spend'] / int_vs_dom_ratios['adjusted_total_domestic_spend']) * 100
int_vs_dom_ratios['abroad_online_ratio'] = (int_vs_dom_ratios['adjusted_online_abroad_spend'] / int_vs_dom_ratios['adjusted_total_abroad_spend']) * 100
# Totals come from section 1 - this won't work if the cell dropping the below columns has been run (also in section 1)
int_vs_dom_ratios['total_online_ratio'] = (adjusted_spend['adjusted_total_online_spend'] / adjusted_spend['adjusted_total_spend']) * 100 


int_vs_dom_ratios

In [ ]:
# Dropping the extra columns so it's just the data for the graph
int_vs_dom_ratios = int_vs_dom_ratios.drop(columns=['total_spend_domestic', 'total_spend_abroad', 'total_online_spend_domestic', 'total_online_spend_abroad', 'adjusted_total_domestic_spend', 'adjusted_total_abroad_spend', 'adjusted_online_domestic_spend', 'adjusted_online_abroad_spend'])

int_vs_dom_ratios

In [ ]:
# Line chart for the second section's graph
fig2 = px.line(
        int_vs_dom_ratios,
        x="time_period_value",
        y=["abroad_online_ratio", "domestic_online_ratio"],
    )
fig2

In [ ]:
# 3. Online spend by country

In [ ]:
# Total abroad spend by UK cardholders by destination country
total_by_country = '''SELECT time_period_value, sum(spend) as total_spend, destination_country
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM'
GROUP BY time_period_value, destination_country
ORDER BY time_period_value ASC'''
spend_by_country = bq.read_bq_table_sql(client, total_by_country)
#spend_by_country

In [ ]:
# Total spend by UK cardholders by destination country, online
online_by_country = '''SELECT time_period_value, sum(spend) as online_spend, destination_country
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM'
GROUP BY time_period_value, destination_country
ORDER BY time_period_value ASC'''
online_spend_by_country = bq.read_bq_table_sql(client, online_by_country)
#online_spend_by_country

In [ ]:
# Summing everything up on a yearly basis
spend_by_country['Year'] = spend_by_country['time_period_value'].str[:4]
yearly_totals = spend_by_country.groupby(['destination_country', 'Year'])['total_spend'].sum().reset_index()

online_spend_by_country['Year'] = online_spend_by_country['time_period_value'].str[:4]
online_yearly_totals = online_spend_by_country.groupby(['destination_country', 'Year'])['online_spend'].sum().reset_index()

online_spend_by_country

In [ ]:
# Merging the 2 quarterly tables together (checked if spend figures are lined up properly, looks good)
merged_quarterly_totals = pd.merge(spend_by_country, online_spend_by_country, on=('time_period_value', 'destination_country'), how='outer')
#merged_quarterly_totals

In [ ]:
# 1. % of spend that was online for each country per quarter
merged_quarterly_totals['percent_online_spend'] = (merged_quarterly_totals['online_spend'] / merged_quarterly_totals['total_spend']) * 100

# 2. % of total online spend that each country contributes in each quarter
# First, calculate total online spend per year
total_online_per_quarter = merged_quarterly_totals.groupby('time_period_value')['online_spend'].transform('sum')

# Then calculate the percentage each country contributes
merged_quarterly_totals['percent_of_total_online_by_country'] = (merged_quarterly_totals['online_spend'] / total_online_per_quarter) * 100
#merged_quarterly_totals.to_csv("quarterly_by_country.csv")

In [ ]:
# Merging the 2 yearly tables together (checked if spend figures are lined up properly, looks good)
merged_yearly_totals = pd.merge(yearly_totals, online_yearly_totals, on=('Year', 'destination_country'), how='outer')
#merged_yearly_totals

In [ ]:
# 1. % of spend that was online for each country-year
merged_yearly_totals['percent_online_spend'] = (merged_yearly_totals['online_spend'] / merged_yearly_totals['total_spend']) * 100

# 2. % of total online spend that each country contributes in each year
# First, calculate total online spend per year
total_online_per_year = merged_yearly_totals.groupby('Year')['online_spend'].transform('sum')

# Then calculate the percentage each country contributes
merged_yearly_totals['percent_of_total_online_by_country'] = (merged_yearly_totals['online_spend'] / total_online_per_year) * 100
merged_yearly_totals.to_csv("temp.csv")

In [ ]:
# Dropping and cleaning the table so it only contains the relevant data for graph
# Only want 2019 and 2024 data so
years_to_remove = ['2020', '2021', '2022', '2023', '2025'] #Number is stored as a string so need to use ''

# Filter the DataFrame
spend_by_country_graph = merged_yearly_totals[~merged_yearly_totals['Year'].isin(years_to_remove)]
spend_by_country_graph = spend_by_country_graph.drop(columns=['total_spend', 'online_spend'])

#spend_by_country_graph

In [ ]:
# Pivot to get 2019 and 2024 values side-by-side
pivot_df = spend_by_country_graph.pivot(index='destination_country', columns='Year', values=['percent_online_spend', 'percent_of_total_online_by_country'])
pivot_df.columns = [f"{col}_{year}" for col, year in pivot_df.columns]
pivot_df.reset_index(inplace=True)
pivot_df

In [ ]:
# Create bubble chart
fig3 = px.scatter(
    pivot_df,
    x='percent_online_spend_2019',
    y='percent_online_spend_2024',
    size='percent_of_total_online_by_country_2024',
    hover_name='destination_country',
    labels={
        'percent_online_spend_2019': '% Online Spend in 2019',
        'percent_online_spend_2024': '% Online Spend in 2024',
        'percent_of_total_online_by_country_2024': '% of Total Online Spend in 2024'
    },
    title='Online Spend Comparison: 2019 vs 2024 by Country'
)

fig3.show()

In [ ]:
# 4. Online spending by MCC

In [ ]:
# Total spend abroad+online by UK cardholders by merchant category
abroad_online_by_mcc = '''SELECT time_period_value, sum(spend) as total_spend_abroad_online, mcc
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter'
  AND mcg != 'All' 
  AND mcc != 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM'
GROUP BY time_period_value, mcc
ORDER BY time_period_value ASC'''
spend_by_mcc = bq.read_bq_table_sql(client, abroad_online_by_mcc)
#spend_by_mcc

In [ ]:
# Summing everything up on a yearly basis
spend_by_mcc['Year'] = spend_by_mcc['time_period_value'].str[:4]
yearly_totals = spend_by_mcc.groupby(['mcc', 'Year'])['total_spend_abroad_online'].sum().reset_index()
yearly_totals.to_csv('spendbymcc.csv')

yearly_totals

In [ ]:
adjusted_by_mcc = yearly_totals

# Merge the two dataframes on 'Year'
merged = pd.merge(adjusted_by_mcc, yearly_cardholders, on='Year', how='left')

# Calculate adjusted spend
merged['adjusted_spend_by_mcc_abroad_online'] = merged['total_spend_abroad_online'] * merged['Change from Base']


adjusted_by_mcc = merged
#adjusted_by_mcc

In [ ]:
# Only need the 2024 data for this diagram so removing the other years
years_to_remove = ['2019', '2020', '2021', '2022', '2023', '2025']

# Filter the DataFrame
adjusted_by_mcc = adjusted_by_mcc[~adjusted_by_mcc['Year'].isin(years_to_remove)]
adjusted_by_mcc = adjusted_by_mcc.drop(columns=['total_spend_abroad_online', 'Change from Base', 'total_cardholders'])

#adjusted_by_mcc

In [ ]:
# Ordering the table by size of spend
spend_by_mcc_ordered = adjusted_by_mcc.sort_values(by='adjusted_spend_by_mcc_abroad_online', ascending=False)

#spend_by_mcc_ordered

In [ ]:
# Calculating the % of the total spend abroad+online in 2024 each mcc makes up
all_mcc_spend_abroad_online = spend_by_mcc_ordered['adjusted_spend_by_mcc_abroad_online'].sum()
# Summing up every MCC does not equal using mcc = 'All' because there is some unknown category of spend due to data suppressios
# We decided that summing each individual mcc group up and using that total to calculate the % would be most appropriate

spend_by_mcc_ordered["% of 'All' mcc spend"] = (spend_by_mcc_ordered['adjusted_spend_by_mcc_abroad_online'] / all_mcc_spend_abroad_online) * 100

#spend_by_mcc_ordered

In [ ]:
# Select for top 10
top_10_mcc = spend_by_mcc_ordered.head(10)

top_10_mcc

In [ ]:
# Creating the bar graph

fig4 = px.bar(top_10_mcc, x="mcc", y="% of 'All' mcc spend", title="Top 10 MCCs by share of Online + Abroad Spending")
fig4.show()

In [ ]:

# 5. International Cardholder Spend Online by Region

In [ ]:
# Total spend in the UK by international cardholders
int_total_spend = '''SELECT time_period_value, sum(spend) as int_total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'International Cardholder'
  AND destination_country = 'UNITED KINGDOM'
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
int_total_spend = bq.read_bq_table_sql(client, int_total_spend)
int_total_spend.head()

In [ ]:
# Total spend in the UK by international cardholders
int_online_spend = '''SELECT time_period_value, sum(spend) as int_online_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'International Cardholder'
  AND destination_country = 'UNITED KINGDOM'
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
int_online_spend = bq.read_bq_table_sql(client, int_online_spend)
int_online_spend.head()

In [ ]:
# Merge the two dataframes on 'time_period_value'
merged = pd.merge(int_total_spend, int_online_spend, on='time_period_value', how='left')
merged

In [ ]:
# Calculating the proportions of each categories spend that is online
int_total_vs_online = merged
int_total_vs_online['Online % Ratio'] = (int_total_vs_online['int_online_spend'] / int_total_vs_online['int_total_spend']) * 100
int_total_vs_online


In [ ]:
# Calculating the indexed spend value for each type of spend
int_total_vs_online['online_spend_index'] = (int_total_vs_online['int_online_spend'] / average_spend(int_total_vs_online, 'int_online_spend')) * 100
int_total_vs_online['total_spend_index'] = (int_total_vs_online['int_total_spend'] / average_spend(int_total_vs_online, 'int_total_spend')) * 100
int_total_vs_online

In [ ]:
int_total_vs_online_graph = int_total_vs_online.drop(columns=['int_total_spend', 'int_online_spend'])
int_total_vs_online_graph

In [ ]:

# Create the figure
fig5 = go.Figure()

# Line for online spend index
fig5.add_trace(go.Scatter(
    x=int_total_vs_online_graph["time_period_value"],
    y=int_total_vs_online_graph["online_spend_index"],
    mode='lines+markers',
    name='Online Spend Index'
))

# Line for total spend index
fig5.add_trace(go.Scatter(
    x=int_total_vs_online_graph["time_period_value"],
    y=int_total_vs_online_graph["total_spend_index"],
    mode='lines+markers',
    name='Total Spend Index'
))

# Bar for Online % Ratio
fig5.add_trace(go.Bar(
    x=int_total_vs_online_graph["time_period_value"],
    y=int_total_vs_online_graph["Online % Ratio"],
    name='Online % Ratio',
    yaxis='y2',
    opacity=0.6
))

# Layout with secondary y-axis
fig5.update_layout(
    title="Spend Indices and Online % Ratio Over Time",
    xaxis_title="Time Period",
    yaxis=dict(
        title="Spend Index",
        side='left'
    ),
    yaxis2=dict(
        title="Online % Ratio",
        overlaying='y',
        side='right',
        range=[0, 100]
    ),
    #legend=dict(x=0.01, y=0.99)
)

fig5.show()
